In [2]:
import cv2
import numpy as np

In [15]:
#image_ = cv2.imread("images.jpg")
image = cv2.VideoCapture(0)
_,image_= image.read()

In [16]:
model = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

In [17]:
with open ("coco.names","r") as f:
    names = f.read().split("\n") 
    
print(names)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']


In [18]:
layer_names = model.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in model.getUnconnectedOutLayers()]

In [19]:
colours = np.random.uniform(0,255,size = (len(names),3))

In [20]:
#image = cv2.resize(image ,(416,416))
print(image_.shape)

(480, 640, 3)


In [21]:
hh,ww,_ = image_.shape

In [22]:
#converting image to binary object
blob_image = cv2.dnn.blobFromImage(image_,
                                  (1 / 255),
                                  (416,416),
                                  (False,False,False),
                                   crop = False)

In [23]:
model.setInput(blob_image)
output = model.forward(output_layers)

In [24]:
print(output)  
print(output[0])

[array([[0.04108663, 0.0442506 , 0.3161306 , ..., 0.        , 0.        ,
        0.        ],
       [0.0560884 , 0.03575155, 0.24507889, ..., 0.        , 0.        ,
        0.        ],
       [0.0524606 , 0.03267331, 0.870302  , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95119774, 0.94871575, 0.3259159 , ..., 0.        , 0.        ,
        0.        ],
       [0.9657825 , 0.96201974, 0.30091906, ..., 0.        , 0.        ,
        0.        ],
       [0.9662715 , 0.9646939 , 0.81547177, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02051784, 0.02204043, 0.05392156, ..., 0.        , 0.        ,
        0.        ],
       [0.01450252, 0.02188254, 0.30220136, ..., 0.        , 0.        ,
        0.        ],
       [0.02444869, 0.02025194, 0.08334242, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97398007, 0.97440135, 0.04774071, ..., 0.        , 0.        ,
        0.        ],
       [0.9802085 

In [25]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in output:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * ww)
            center_y = int(detection[1] * hh)
            w = int(detection[2] * ww)
            h = int(detection[3] * hh)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [26]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(names[class_ids[i]])
        color = colours[i]
        cv2.rectangle(image_, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image_, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", image_)
cv2.waitKey(0)
cv2.destroyAllWindows()
image.release()